In [1]:
import numpy as np
import numba
import itertools
import sys

# Speed/Memory Tradeoff for using Python ints vs. Numpy.uint8

In [56]:
%timeit 5

6.36 ns ± 0.086 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [57]:
sys.getsizeof(5)

28

In [58]:
%timeit np.uint8(5)

260 ns ± 5.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
sys.getsizeof(np.uint8(5))

25

All those bytes are going to add up, but no one will ever notice the time.

In [60]:
%timeit 2 + 2

9.33 ns ± 0.231 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [61]:
%timeit 2 + np.uint(2)

3.28 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [78]:
%timeit np.uint8(2) + np.uint8(2)

565 ns ± 11.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Pretty sure addition will never be done but clearly arithmetic operations are better with with `np.uint8`'s

# Initial card array building

In [69]:
%timeit np.array([0,0,0,0,0], dtype=np.uint8)

1.78 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [71]:
%timeit np.array([0,0,0,0,0], dtype=int)

1.66 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%timeit np.zeros(5, dtype=np.uint8)

1.08 µs ± 5.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%timeit np.zeros(np.uint(5), dtype=np.uint8)

1.6 µs ± 6.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [63]:
%timeit np.zeros(np.uint(5), dtype=int)

1.51 µs ± 9.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [64]:
%timeit np.zeros(5, dtype=int)

999 ns ± 7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


There is no point in using `np.uint8`'s as arguments.

In [76]:
sys.getsizeof(np.zeros(5, dtype=np.uint8))

101

In [77]:
sys.getsizeof(np.zeros(5, dtype=int))

136

I'd say the size difference of the different int types is worth the ~600 nanosecond difference in building time.

# Hashing

In [137]:
np.random.seed(12341234)
arr = np.random.randint(low=1, high=53, size=5)

In [138]:
%timeit hash(str(arr))

43.9 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [139]:
%timeit hash(arr.tostring())

170 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [140]:
%timeit hash(str(arr.data))

926 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [141]:
%timeit hash(str(arr.data.tobytes))

1.21 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


`.tostring()` is the winner.

## Prove completely uncolliding hashes

In [43]:
combo_dict = {}

In [44]:
cards = [0] * 5 + list(range(1, 53))

In [45]:
possible_hands_with_repeats = itertools.combinations(cards, 5)

In [46]:
possible_hands = set(possible_hands_with_repeats)

In [47]:
for hand in possible_hands:
    curr_hand = np.array(hand, dtype=np.uint8)
    curr_hash = hash(curr_hand.tostring())
    if curr_hash in combo_dict:
        print(f"already in: {curr_hash}, {combo_dict[curr_hash]}")
        print(f"to insert: {curr_hash}, {curr_hand}")
        break
    else:
        combo_dict[curr_hash] = curr_hand

# Misc

In [73]:
np.array(list(itertools.combinations([1, 2, 3, 4], 2)))

array([[1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4],
       [3, 4]])

In [84]:
a = []
a.extend(itertools.combinations([1, 2, 3, 4], 2))
a.extend(itertools.combinations([5, 6, 7, 8], 2))
a

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (5, 6),
 (5, 7),
 (5, 8),
 (6, 7),
 (6, 8),
 (7, 8)]

In [89]:
cards_but_not_hand = []
suits = np.arange(1, 53, dtype=np.uint8).reshape(13, 4)
for suit in suits:
    cards_but_not_hand.extend(itertools.combinations(suit, 2))

In [2]:
%timeit sum(range(1, 13))

320 ns ± 2.87 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [3]:
%timeit np.sum(np.arange(1, 13))

5.1 µs ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
2 * sum(range(1, 13))

156

In [5]:
from scipy.special import comb

In [8]:
comb(13, 2, exact=True)

78

In [132]:
np.array((1, 2, 3, 4, 5), dtype=np.uint8)

array([1, 2, 3, 4, 5], dtype=uint8)

In [113]:
a = itertools.combinations(range(1, 5), 2)

In [114]:
b = itertools.combinations(range(5, 9), 3)

In [115]:
c = itertools.product(a, b)

In [116]:
func = lambda dub_trip: dub_trip[0] + dub_trip[1]

In [117]:
d = list(map(func, c))

In [118]:
%timeit np.array(d, dtype=np.uint8)

10.8 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
import numpy as np
from itertools import combinations as it_comb, product as it_prod
from hand_hash_generator import add_to_combo_dict, cards, suits
from numba import jit, void

In [21]:
combo_dict = {}

jit(void())
def fullhouse():
    """
    adds all fullhouse hands to the combo dict
    """
    # combines double triples or triple doubles and adds to combo dict
    def combine_and_add(combos):
        hand = np.array(combos[0] + combos[1], dtype=np.uint8)
        add_to_combo_dict(hand, 3)

    for suit1, suit2 in it_comb(suits, 2):
        # double triples
        base_doubles = it_comb(suit1, 2)
        add_triples = it_comb(suit2, 3)
        dub_trip = it_prod(base_doubles, add_triples)
        for d_t in dub_trip:
            combine_and_add(d_t)

        # triple doubles
        base_triples = it_comb(suit1, 3)
        add_doubles = it_comb(suit2, 2)
        trip_dub = it_prod(base_triples, add_doubles)
        for t_d in trip_dub:
            combine_and_add(t_d)
            
fullhouse()

In [22]:
combo_dict

{}

In [23]:
import deepdish as dd

In [25]:
combo_dict = dd.io.load("combo_dict.h5")

In [26]:
len(combo_dict)

3744

In [28]:
from scipy.special import comb
2 * comb(13, 2, exact=True) * comb(4, 3, exact=True) * comb(4, 2, exact=True)

3744

In [3]:

import numba
numba.__version__

'0.37.0'

In [11]:
from numba import jit

@jit
def foo():
    def bar():
        pass
    pass

foo()

LoweringError: Failed at object (object mode backend)
make_function(name=$const0.2, code=<code object bar at 0x7f335877f780, file "<ipython-input-11-3fbbde7cf73c>", line 5>, closure=None, defaults=None)
File "<ipython-input-11-3fbbde7cf73c>", line 5
[1] During: lowering "$0.3 = make_function(name=$const0.2, code=<code object bar at 0x7f335877f780, file "<ipython-input-11-3fbbde7cf73c>", line 5>, closure=None, defaults=None)" at <ipython-input-11-3fbbde7cf73c> (5)

In [42]:
from numba import prange, uint8

In [43]:
prange(10)

range(0, 10)

In [44]:
@jit(void(uint8[:, :], uint8))
def add_to_combo_dict_loop(hands, combo):
    for hand in hands:
        add_to_combo_dict(hand, combo)

In [49]:
@jit(void(), parallel=True)
def single():
    """
    adds all single hands to the combo dict
    """
    number_of_singles = 52
    singles = np.zeros(shape=(number_of_singles, 5), dtype=np.uint8)
    singles[:, 4] = np.arange(1, 53)
    add_to_combo_dict_loop(singles, 1)

In [50]:
@jit(void(), parallel=True)
def single1():
    """
    adds all single hands to the combo dict
    """
    number_of_singles = 52
    singles = np.zeros(shape=(number_of_singles, 5), dtype=np.uint8)
    singles[:, 4] = prange(1, 53)
    add_to_combo_dict_loop(singles, 1)

In [51]:
%timeit single()

1.8 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
%timeit single1()

1.77 ms ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
